In [1]:
from pathlib import Path
import pickle
from collections import defaultdict
import sys 
# Add the base_agent directory to sys.path
base_agent_dir = '/vol/biomedic3/bglocker/ugproj2324/nns20/cxr-agent/base_agent'
sys.path.insert(0, str(base_agent_dir))

from pathology_sets import Pathologies
from pathology_detector import CheXagentVisionTransformerPathologyDetector
from phrase_grounder import BioVilTPhraseGrounder
from generation_engine import GenerationEngine, Llama3Generation, CheXagentLanguageModelGeneration, GeminiFlashGeneration
# USER_PROMPT = "Generate the findings section of the radiology report for the scan"
USER_PROMPT = "What are the findings?"
USER_PROMPT = "Generate the findings section ONLY of the radiology report for this scan"
PATHOLOGY_DETECTION_THRESHOLD = 0.4
PHRASE_GROUNDING_THRESHOLD = 0.2
IGNORE_PATHOLOGIES = {"Support Devices"}
DO_NOT_LOCALISE = {"Cardiomegaly"}
CHEXPERT = True
FILE_NAME = "findings_section_of_report"
SHUFFLE = False
DEVICE = None #"cuda:1"   

IMAGE_DIR_PREFIX = Path("/vol/biodata/data/chest_xray/mimic-cxr-jpg/files/")

In [2]:
pathology_detector = CheXagentVisionTransformerPathologyDetector(pathologies=Pathologies.CHEXPERT, device=DEVICE)
phrase_grounder = BioVilTPhraseGrounder(detection_threshold=PHRASE_GROUNDING_THRESHOLD, device = DEVICE)
l3 = Llama3Generation(device = DEVICE)
cheXagent_lm = CheXagentLanguageModelGeneration(pathology_detector.processor, pathology_detector.model, pathology_detector.generation_config, pathology_detector.device, pathology_detector.dtype)
gemini = GeminiFlashGeneration()


GPU 0: NVIDIA RTX 6000 Ada Generation, Free memory: 48643 MB
GPU 1: NVIDIA RTX 6000 Ada Generation, Free memory: 41465 MB
Selecting GPU 0 with 48643 MB free memory, Device = cuda:0


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'CXRBertTokenizer'.


CheXagent Model loaded


You are using a model of type bert to instantiate a model of type cxr-bert. This is not supported for all configurations of models and can yield errors.
/vol/biomedic3/bglocker/ugproj2324/nns20/cxr-agent/.venv/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at microsoft/BiomedVLP-BioViL-T were not used when initializing CXRBertModel: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing CXRBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This I

Using downloaded and verified file: /tmp/biovil_t_image_model_proj_size_128.pt
GPU 0: NVIDIA RTX 6000 Ada Generation, Free memory: 31391 MB
GPU 1: NVIDIA RTX 6000 Ada Generation, Free memory: 41465 MB
Selecting GPU 1 with 41465 MB free memory, Device = cuda:1
GPU 0: NVIDIA RTX 6000 Ada Generation, Free memory: 31391 MB
GPU 1: NVIDIA RTX 6000 Ada Generation, Free memory: 40463 MB
Selecting GPU 1 with 40463 MB free memory, Device = cuda:1


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
subject_to_image_path = {}
# Read the data file into a list of dictionaries
def read_data_file(cheXpert = CHEXPERT):

    subjects = []
    if cheXpert:
        cheXpert_small_test_path = Path("/vol/biomedic3/bglocker/ugproj2324/nns20/datasets/CheXpert/small/test")
        cheXpert_test_path = Path("/vol/biodata/data/chest_xray/CheXpert-v1.0-small/CheXpert-v1.0-small/test")

        cheXpert_subjects = Path("/vol/biomedic3/bglocker/ugproj2324/nns20/cxr-agent/evaluation_datasets/CheXpert/chexpert_test_random_written_pathologies")

        for line in cheXpert_subjects.read_text().splitlines():
            subject = line.split(",")[0]
            subjects.append(subject)
            subject_path_jpg = cheXpert_test_path / subject
            subject_to_image_path[subject] = subject_path_jpg      
    
    else:
        mimic_cxr_jpg_path = Path('/vol/biodata/data/chest_xray/mimic-cxr-jpg')
        # subject_with_single_scan_no_prior_path = Path("/vol/biomedic3/bglocker/ugproj2324/nns20/cxr-agent/frontend/subjects_with_single_scan_no_prior.txt")
        subjects_for_eval = Path("/vol/biomedic3/bglocker/ugproj2324/nns20/cxr-agent/evaluation_datasets/MIMIC-CXR/mimic_ii_subjects_for_eval.txt")
        for line in subjects_for_eval.read_text().splitlines():
            # two cases: 1) subject with single study 2) subject with multiple studies
            if line[0] == "p":
                parts = line.split("/")
                subject = parts[0][1:] # remove the 'p' from the subject
                subjects.append(subject) 
                study = parts[1][1:]

                subject_path_jpg = mimic_cxr_jpg_path/"files"/f"p{subject[:2]}"/f"p{subject}"/f"s{study}"
                jpg_file = list(subject_path_jpg.glob('*.jpg'))[0]

            else:
                subject = line
                subjects.append(subject)
                subject_path_jpg = mimic_cxr_jpg_path/"files"/ f"p{subject[:2]}/p{subject}"
                
                study_folder = list(subject_path_jpg.glob('*'))[0]
                jpg_file = list(study_folder.glob('*.jpg'))[0]
                
            subject_to_image_path[subject] = jpg_file
            
    return subjects

subjects = read_data_file()

In [4]:
image_path_to_model_responses = defaultdict(dict)

for subject in subjects:
    image_path = subject_to_image_path[subject]

    pathology_confidences, localised_pathologies, chexagent_e2e = GenerationEngine.detect_and_localise_pathologies(
        image_path=image_path,
        pathology_detector=pathology_detector,
        phrase_grounder=phrase_grounder,
        pathology_detection_threshold = PATHOLOGY_DETECTION_THRESHOLD,
        ignore_pathologies=IGNORE_PATHOLOGIES,
        do_not_localise=DO_NOT_LOCALISE,
        prompt_for_chexagent_lm_output= USER_PROMPT,
    )

    model_outputs = {}
    model_outputs['chexagent'] = chexagent_e2e

    gemini_system_prompt, gemini_image_context_prompt = gemini.generate_prompts(pathology_confidences, localised_pathologies)
    model_outputs['gemini_agent'] = gemini.generate_model_output(gemini_system_prompt, gemini_image_context_prompt, user_prompt=USER_PROMPT)

    system_prompt, image_context_prompt = GenerationEngine.generate_prompts(pathology_confidences, localised_pathologies)
    model_outputs['chexagent_agent'] = cheXagent_lm.generate_model_output(system_prompt, image_context_prompt, user_prompt=USER_PROMPT)

    system_prompt, image_context_prompt = l3.generate_prompts(pathology_confidences, localised_pathologies)
    model_outputs['llama3_agent'] = l3.generate_model_output(system_prompt, image_context_prompt, user_prompt=USER_PROMPT)

    image_path_to_model_responses[image_path] = model_outputs


/vol/biomedic3/bglocker/ugproj2324/nns20/cxr-agent/.hf_cache/modules/transformers_modules/4934e91451945c8218c267aae9c34929a7677829/processing_chexagent.py:86: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(pixel_values) for pixel_values in encoding_image_processor["pixel_values"]]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

In [5]:
# save image_path_to_model_responses as a pickle file
if CHEXPERT:
    output_file = Path(f"/vol/biomedic3/bglocker/ugproj2324/nns20/cxr-agent/frontend/stored_responses/CheXpert/{FILE_NAME}.pkl")
else:
    output_file = Path(f"/vol/biomedic3/bglocker/ugproj2324/nns20/cxr-agent/frontend/stored_responses/MIMIC-CXR/{FILE_NAME}.pkl")

with open(output_file, 'wb') as f:
    pickle.dump(image_path_to_model_responses, f)

# # read the pickle file
# with open(output_file, 'rb') as f:
#     image_path_to_model_responses = pickle.load(f)
#     print(image_path_to_model_responses)
